In [ ]:
!pip install tensorflow


In [ ]:
import tensorflow as tf  # Framework utama untuk deep learning
from tensorflow.keras.models import Sequential  # Untuk membuat model neural network
from tensorflow.keras.layers import Dense, Dropout  # Untuk layer dense dan regulasi
from tensorflow.keras.utils import to_categorical  # Untuk one-hot encoding

import pandas as pd             # Untuk manipulasi data
import numpy as np              # Operasi numerik
from sklearn.model_selection import train_test_split  # Membagi data latih dan uji
from sklearn.preprocessing import LabelEncoder, MinMaxScaler  # Encoding label & normalisasi


In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Path ke file
file_path = '/content/drive/My Drive/semester 5/AI/tugas akhir/soil.csv'

# Baca file CSV
df = pd.read_csv(file_path)
print(df.head())


    N   P   K  temperature   humidity        ph    rainfall label
0  90  42  43    20.879744  82.002744  6.502985  202.935536  rice
1  85  58  41    21.770462  80.319644  7.038096  226.655537  rice
2  60  55  44    23.004459  82.320763  7.840207  263.964248  rice
3  74  35  40    26.491096  80.158363  6.980401  242.864034  rice
4  78  42  42    20.130175  81.604873  7.628473  262.717340  rice


In [ ]:
df = df.drop_duplicates()

In [ ]:
print(df.describe())

                 N            P            K  temperature     humidity  \
count  2200.000000  2200.000000  2200.000000  2200.000000  2200.000000   
mean     50.551818    53.362727    48.149091    25.616244    71.481779   
std      36.917334    32.985883    50.647931     5.063749    22.263812   
min       0.000000     5.000000     5.000000     8.825675    14.258040   
25%      21.000000    28.000000    20.000000    22.769375    60.261953   
50%      37.000000    51.000000    32.000000    25.598693    80.473146   
75%      84.250000    68.000000    49.000000    28.561654    89.948771   
max     140.000000   145.000000   205.000000    43.675493    99.981876   

                ph     rainfall  
count  2200.000000  2200.000000  
mean      6.469480   103.463655  
std       0.773938    54.958389  
min       3.504752    20.211267  
25%       5.971693    64.551686  
50%       6.425045    94.867624  
75%       6.923643   124.267508  
max       9.935091   298.560117  


In [ ]:
# Seleksi kolom yang dibutuhkan
df_selected = df[['N','P','K', 'humidity', 'ph', 'label']]
print(df_selected.head())





    N   P   K   humidity        ph label
0  90  42  43  82.002744  6.502985  rice
1  85  58  41  80.319644  7.038096  rice
2  60  55  44  82.320763  7.840207  rice
3  74  35  40  80.158363  6.980401  rice
4  78  42  42  81.604873  7.628473  rice


In [ ]:
# Buat salinan aman dari dataset
df_selected = df[['N','P','K', 'humidity', 'ph', 'label']].copy()

# Periksa kategori dalam kolom label
print("Unique labels:", df_selected['label'].unique())

# Lakukan encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_selected['label'] = le.fit_transform(df_selected['label'])

print(df_selected.head())


Unique labels: ['rice' 'maize' 'chickpea' 'kidneybeans' 'pigeonpeas' 'mothbeans'
 'mungbean' 'blackgram' 'lentil' 'pomegranate' 'banana' 'mango' 'grapes'
 'watermelon' 'muskmelon' 'apple' 'orange' 'papaya' 'coconut' 'cotton'
 'jute' 'coffee']
    N   P   K   humidity        ph  label
0  90  42  43  82.002744  6.502985     20
1  85  58  41  80.319644  7.038096     20
2  60  55  44  82.320763  7.840207     20
3  74  35  40  80.158363  6.980401     20
4  78  42  42  81.604873  7.628473     20


In [ ]:
# Sampling 50% dari data asli
df_sampled = df_selected.sample(frac=0.9, random_state=42)

# Pisahkan fitur dan target dari data yang sudah disampling
X = df_sampled.drop('label', axis=1)
y = df_sampled['label']

# Split data menjadi train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cek jumlah data
print("Data latih:", X_train.shape)
print("Data uji:", X_test.shape)


Data latih: (1584, 5)
Data uji: (396, 5)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
import pandas as pd

# Pastikan X_train dan y_train adalah NumPy array
X_train_np = X_train  # Tidak perlu konversi karena sudah NumPy array
y_train_np = y_train

# Buat DataFrame dari X_train dan tambahkan kolom 'label'
train_data = pd.DataFrame(X_train_np, columns=[f"feature_{i+1}" for i in range(X_train_np.shape[1])])
train_data['label'] = y_train_np

print(train_data.head())


   feature_1  feature_2  feature_3  feature_4  feature_5  label
0   0.600000   0.221429      0.115   0.481705   0.406688   20.0
1   0.285714   0.450000      0.060   0.593659   0.657570    NaN
2   0.635714   0.392857      0.070   0.611641   0.374617   20.0
3   0.214286   0.835714      0.960   0.915293   0.321677   20.0
4   0.771429   0.292857      0.060   0.823792   0.533014   20.0


In [ ]:

# Normalisasi fitur
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define the neural network model
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),  # First hidden layer
    Dropout(0.5),  # Dropout at 50%
    Dense(64, activation='relu'),                               # Second hidden layer
    Dropout(0.3),  # Dropout at 30%
    Dense(len(y.unique()), activation='softmax')                # Output layer
])

# Compile the model
model.compile(
    optimizer='adam',                 # Adam optimizer
    loss='sparse_categorical_crossentropy',  # Loss function for multi-class classification
    metrics=['accuracy']              # Metric to evaluate models
)
from tensorflow.keras.callbacks import EarlyStopping

early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    patience=10,  # Number of epochs to wait if no improvement and then stop the training
    restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored quantity.

)

# Fit the model on the training data
history = model.fit(
    X_train, y_train,
    validation_split=0.2,  # Use 20% of the training data for validation
    epochs=500,             # Number of iterations over the entire dataset
    batch_size=32          # Number of samples per gradient update
)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc*100:.2f}%")


Epoch 1/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.0572 - loss: 3.0751 - val_accuracy: 0.1420 - val_loss: 2.9943
Epoch 2/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1528 - loss: 2.9685 - val_accuracy: 0.2240 - val_loss: 2.8054
Epoch 3/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1863 - loss: 2.7820 - val_accuracy: 0.2902 - val_loss: 2.5159
Epoch 4/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2540 - loss: 2.4936 - val_accuracy: 0.4669 - val_loss: 2.1872
Epoch 5/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2615 - loss: 2.2356 - val_accuracy: 0.5016 - val_loss: 1.9047
Epoch 6/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3565 - loss: 2.0007 - val_accuracy: 0.5868 - val_loss: 1.6916
Epoch 7/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3635 - loss: 1.8173 - val_accuracy: 0.6309 - val_loss: 1.5069
Epoch 8/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4256 - loss: 1.6937 - val_accuracy: 0.6877 - v

In [ ]:
model.save('C:/Users/user/Documents/my_model.h5')
